### Stock sentiment using News headlines

In [1]:
"""
****** Imports *******
"""
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
data = pd.read_csv("dataset/News_headline.csv",encoding="utf-8")
data.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."


In [3]:
""" 
TEXT PRE-PROCESSING
"""
## Remove b from start & remove everything except string
data.replace("[b][^a-zA-Z]"," ",regex=True,inplace=True)
## Remove commas from text
data.replace("'","",regex=True,inplace=True)

In [5]:
## TRAIN-TEST SPLIT
train = data[data['Date'] < '20150101']
test = data[data['Date'] > '20141231']

In [10]:
data = train.iloc[:,2:27]

## Renaming cols for ease of access
newIndex = [str(i) for i in range(25)]
data.columns = newIndex

## Converting text to lower-Case
for index in newIndex:
    data[index] = data[index].str.lower()
data.head()

In [21]:
"""
To apply NLP, we have to convert headline of every column of a row into a single text
i.e ''.join(str(x) for x in data.iloc[1,:])

-> len(data.index) = Total Number of rows
"""
headlines = []

for row in range(0,len(data.index)):
    headlines.append(''.join(str(x) for x in data.iloc[row,:]))

In [25]:
"""
Let's apply NLP
"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [26]:
## Bag of words 
countVector = CountVectorizer(ngram_range=(2,2))
train_data = countVector.fit_transform(headlines)

In [28]:
## Implement Random Classifier
rc = RandomForestClassifier(n_estimators=200,criterion='entropy')
rc.fit(train_data,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [29]:
## Validation
test_transform = []

for row in range(0,len(test.index)):
    test_transform.append(''.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countVector.transform(test_transform)
predictions = rc.predict(test_dataset)

In [31]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix = confusion_matrix(test['Label'],predictions)
score = accuracy_score(test['Label'],predictions)
report = classification_report(test['Label'],predictions)

In [33]:
print("Confusion Matrix:")
print(matrix)

Confusion Matrix:
[[133  53]
 [  2 190]]


In [35]:
print(f"Accuracy: {score}")

Accuracy: 0.8544973544973545


In [40]:
print("Classification Report:\n")

print(report)

Classification Report:

              precision    recall  f1-score   support

           0       0.99      0.72      0.83       186
           1       0.78      0.99      0.87       192

    accuracy                           0.85       378
   macro avg       0.88      0.85      0.85       378
weighted avg       0.88      0.85      0.85       378

